In [1]:
# CONECTANDO AO GOOGLE CLOUD STORAGE (GCS)
PROJECT = 'mapbiomas'

from google.colab import auth
from google.cloud import storage
auth.authenticate_user()

!gcloud config set project {PROJECT}

Updated property [core/project].


In [ ]:
# CONECTANDO AO GOOGLE EARTH ENGINE (GEE)
import ee
# ee.Authenticate()
ee.Initialize()

In [3]:
gcs_folder = 'gs://shared-development-storage/AUXILIARES/MAPBIOMAS-AR/'
url_gcs_folder = 'https://storage.cloud.google.com/shared-development-storage/AUXILIARES/MAPBIOMAS-AR/'

In [5]:
import os
import sys

from glob import glob
import os
from bs4 import BeautifulSoup
import requests
from pathlib import Path

In [11]:
url_floresta = 'http://floresta.if.usp.br/ftp/public/Temp/MapbiomasAR/CH4_shapefiles'

def listFD(url):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    return [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href')]

obj_list = []

urls = listFD(url_floresta)
urls = list(filter(lambda f: f.find("?C=") <= 0, urls))
urls = urls[1:]

print(urls)

['http://floresta.if.usp.br/ftp/public/Temp/MapbiomasAR/CH4_shapefiles/grib2test.cpg', 'http://floresta.if.usp.br/ftp/public/Temp/MapbiomasAR/CH4_shapefiles/grib2test.dbf', 'http://floresta.if.usp.br/ftp/public/Temp/MapbiomasAR/CH4_shapefiles/grib2test.prj', 'http://floresta.if.usp.br/ftp/public/Temp/MapbiomasAR/CH4_shapefiles/grib2test.shp', 'http://floresta.if.usp.br/ftp/public/Temp/MapbiomasAR/CH4_shapefiles/grib2test.shx', 'http://floresta.if.usp.br/ftp/public/Temp/MapbiomasAR/CH4_shapefiles/xch4_merged_mips2003_1.cpg', 'http://floresta.if.usp.br/ftp/public/Temp/MapbiomasAR/CH4_shapefiles/xch4_merged_mips2003_1.dbf', 'http://floresta.if.usp.br/ftp/public/Temp/MapbiomasAR/CH4_shapefiles/xch4_merged_mips2003_1.prj', 'http://floresta.if.usp.br/ftp/public/Temp/MapbiomasAR/CH4_shapefiles/xch4_merged_mips2003_1.shp', 'http://floresta.if.usp.br/ftp/public/Temp/MapbiomasAR/CH4_shapefiles/xch4_merged_mips2003_1.shx', 'http://floresta.if.usp.br/ftp/public/Temp/MapbiomasAR/CH4_shapefiles/xch4

In [21]:

for url in urls[5:]:
  data = url.split('/')[8]

  r = requests.get(url, allow_redirects=True)
  input_address = '/content/'+data

  open(input_address, 'wb').write(r.content)

  folder_gcs = 'gs://shared-development-storage/AUXILIARES/MAPBIOMAS-AR'
  cmd_gcs = 'gsutil cp {0} {1}/{2}'.format(input_address, folder_gcs, data)
  os.system(cmd_gcs)
  # print(cmd_gcs)

  if data[-4:] == '.shx':
    
    asset_id = 'projects/mapbiomas-workspace/AR/CH4_VECTORS'
    cmd = 'earthengine upload table --asset_id={0}/{1} {2}/{1}.shp'.format(asset_id,data[:-4],folder_gcs)
    os.system(cmd)
    # print(cmd)
  
  print('INFO: {0}/{1}  --> {2}'.format((urls.index(url) + 1),len(urls), cmd))

INFO: 6/1050  --> earthengine upload table --asset_id=projects/mapbiomas-workspace/AR/CH4_VECTORS/grib2test gs://shared-development-storage/AUXILIARES/MAPBIOMAS-AR/grib2test.shp
INFO: 7/1050  --> earthengine upload table --asset_id=projects/mapbiomas-workspace/AR/CH4_VECTORS/grib2test gs://shared-development-storage/AUXILIARES/MAPBIOMAS-AR/grib2test.shp
INFO: 8/1050  --> earthengine upload table --asset_id=projects/mapbiomas-workspace/AR/CH4_VECTORS/grib2test gs://shared-development-storage/AUXILIARES/MAPBIOMAS-AR/grib2test.shp
INFO: 9/1050  --> earthengine upload table --asset_id=projects/mapbiomas-workspace/AR/CH4_VECTORS/grib2test gs://shared-development-storage/AUXILIARES/MAPBIOMAS-AR/grib2test.shp
INFO: 10/1050  --> earthengine upload table --asset_id=projects/mapbiomas-workspace/AR/CH4_VECTORS/xch4_merged_mips2003_1 gs://shared-development-storage/AUXILIARES/MAPBIOMAS-AR/xch4_merged_mips2003_1.shp
INFO: 11/1050  --> earthengine upload table --asset_id=projects/mapbiomas-workspace